In [1]:
%load_ext line_profiler
import os, re
import sys
import glob
import datetime as dt
from pathlib import Path
from subprocess import run
from typing import Union
from time import strftime, strptime

from dateutil.parser import parse
from numexpr import evaluate
import numpy as np
import pandas as pd
import pickle as pkl
import logging
from datefinder import find_dates

In [206]:
#총 5가지 경우
# YYYYMMDD, YYMMDD, YYYYMM, YYYY, MMDD
# 남은 과제 : format(%m%d)일 때 현재 년도 기준으로 년도를 붙여주는 방법 생각해보기
#           format(%Y%m)과 format(%y%m%d) 구분할 방법 생각해보기
### 
# 기본적으로 한국식 (연-월-일) 표기라고 가정한다.
# 1. 입력을 %d가 아닌 모든 기호로 split한다.
## 경우의 수가 몇 가지 있다.
## 1. len(splitted_input) == 1
### 이 경우는 1. 8자리 혹은 6자리를 입력받았을 때 --> 연, 월, 일을 입력받았다.
###         2. 4자리 혹은 2자리를 입력받았을 때 --> 년도를 입력받았다. 로 인식한다.
## 2. len(splitted_input) == 2
### 이 경우는 1. 첫 번째 인자가 연, 두 번째 인자가 월인 경우
###         2. 첫 번째 인자가 월, 두 번째 인자가 일인 경우를 구별하여 인식한다.
## 3. len(splitted_input) == 3 
### 이 경우는 연-월-일을 입력받은 것으로 인식한다.

In [2]:
def set_logger(level=logging.INFO, name=__name__):
    logger = logging.getLogger(name)
    logger.setLevel(level)
    handler = logging.FileHandler('log_file.log')
    formatter = logging.Formatter(
        '%(asctime)s : %(name)s  : %(funcName)s : %(levelname)s : %(message)s')
    handler.setFormatter(formatter)
    logger.addHandler(handler)
    return logger

In [3]:
# PATH = os.path.abspath(r"../data/10X/cleaned/1998/QTR2/") # on home
# PATH = os.path.abspath(r"C:\Users\wonhyeong\workings\data\10X\cleaned") # on office
PATH = os.path.abspath(r"/Users/jowonhyeong/Desktop/workspace/data")  # on mac

In [13]:
# make sample dataframe
df: pd.DataFrame = pd.DataFrame(np.random.randint(0, 100, size=(10000, 4)), columns=list('ABCD'))
# df to {col1: df.groupby('col1').get_group(x), col2: df.groupby('col2').get_group(x), ...}
df_dict = {col: df.groupby(col).groups for col in df.columns}
idx_dict = df.groupby(by='A').apply(lambda x: x.index.tolist()).to_dict()

In [16]:
df.loc[df_dict['A'][11], 'B']

41      23
65      36
205     87
281     18
306      6
        ..
9658    19
9757    26
9806    86
9886    35
9897    31
Name: B, Length: 106, dtype: int64

In [39]:
class spider:
    # module dependencies
    import datetime as dt
    import os
    from pathlib import Path
    from dateutil.parser import parse

    import numpy as np
    import pandas as pd
    from numexpr import evaluate as ne_eval

    class phrase:
        def __init__(self, keyword: str):
            self.phrase = self._preprocessor(keyword)
            # assert self.key or self.modifier, "There's something wrong with your input"
            return

        def _preprocessor(self, keyword: str):
            # params : string for search
            # return : list of keywords + tags
            keys = ['cik', 'date', 'type', 'name', 'ticker', 'exchange', 'path']
            modifiers = ['after', 'before', 'year', 'qtr']
            keywords = keyword.split(' ')
            tagged = [x for x in keywords if ':' in x]
            untagged = (x for x in keywords if ':' not in x)
            untagged = [self._auto_tagging(x) for x in untagged]
            untagged = [x for x in untagged if x]
            tagged.extend(untagged)
            key = {k: v for k, v in map(lambda x: x.split(':'), tagged) if k in keys}
            if 'cik' in key:
                key['cik'] = int(key['cik'])
            modifier = {k: v for k, v in map(lambda x: x.split(':'), tagged) if k in modifiers}
            modifier = {k: self._date_parser(v) for k, v in modifier.items()}
            return key, modifier

        def _date_parser(self, input: str):
            """확실히 깔끔하게 다듬을 필요가 있음"""
            month = tuple(map(lambda x: str(x).rjust(2, '0') , range(1,13)))
            # check input format
            assert isinstance(input, str), 'input must be a string'
            # check divier in input
            special_chars = set(re.findall(r'[^a-zA-Z0-9]', input))
            if not special_chars:
                if len(input) == 8:
                    return input
                elif len(input) == 6:
                    format = '%y%m%d'
                elif len(input) == 4:
                    if input.startswith(('19', '20')):
                        format = '%Y'
                    elif input.startswith(month):
                        # in current this method is not working, base year is 1900
                        format = '%m%d'
                    else:
                        raise ValueError('Invalid date format')
                else:
                    raise ValueError('Invalid date format')
                date = strptime(input, format)
                return strftime('%Y%m%d', date)
            # check there is only one type of divider
            divier, *_ = special_chars
            assert not _, 'More than one special character found'
            date = find_dates(input, first='year')
            parsed = input.split(divier)
            if len(parsed) == 2:
                if len(parsed[0]) == 4:
                    date = find_dates(input, first='year')
                    date = next(date).strftime('%Y%m01')
                else:
                    date = find_dates(input, first='month')
                    date = next(date).strftime('%Y%m%d')
            elif len(parsed) == 3:
                    date = find_dates(input, first='year')
                    date = next(date).strftime('%Y%m%d')
            else:
                raise ValueError('Invalid date format')
            return date
            
        def _auto_tagging(self, input: str) -> str:
            key = None
            acc = re.compile(r'\d{10}\-\d{2}\-\d{6}')
            form_type = re.compile(r'\d{2}\-\w+')
            if input.isdigit():
                key = 'cik'
            elif input.isalpha():
                if input.isupper():
                    key = 'ticker'
                elif input == ('Nasdaq' or 'NYSE'):
                    key = 'exchange'
            elif re.match(acc, input):
                key = 'acc'
            elif re.match(form_type, input):
                key = 'type'
            if key:
                return f'{key}:{input}'
            return 

    def __init__(self, data_path, index_path='index.pkl'):
        # self.index.cols = ['acc', 'cik', 'date', 'type', 'name', 'ticker', 'exchange', 'path']
        self.dir: Path = Path(data_path)
        self.index_path: Path = self.dir / index_path
        self.index: pd.DataFrame = pd.read_pickle(self.index_path) if (
            self.index_path).exists() else None
        self.index_dict: dict = {col: self.index.groupby(col).groups for col in self.index.columns}
        self.columns: list = self.index.columns
        # There must be a much more elegant way
        self.tree = None
        self._build_tree()
        return

    def search(self, keyword: Union[str, phrase] = '', output: str = 'cik') -> pd.Series:
        """get string of search query and return result of query as pd.Series[idx = acc, val = output]"""
        if isinstance(keyword, str):
            key, modifier = self.phrase(keyword).phrase
        elif isinstance(keyword, spider.phrase):
            key, modifier = keyword.phrase
        else :
            raise ValueError("keyword must be str or phrase")
        assert output in self.columns, f"output must be one of {self.columns}"
        # get spider.syntax object from keywords
        # matrix of search results
        # maybe there's a better way to do this
        matrix: list = []
        for idx, (k, v) in enumerate(key.items()):
            matrix.append(self.index_dict[k][v])
        for idx, (k, v) in enumerate(modifier.items()):
            matrix.append(self._query_as_mod(k, v))
        # logical AND operation, get index of searching files
        # get intersection of all lists
        # if there is no intersection, return empty list
        if not matrix:
            return []
        result: set = set(matrix[0])
        for idx, m in enumerate(matrix):
            result = result.intersection(m)
        # get output of searching files
        result = list(result)
        return self.index.loc[result, output]

    def company(self, after=0, before=99999999, min_freq: int=0, get_period=True):
        """
        search unique cik from given period and return that (cik, *period) to spider.pharse object
        options: min_freq = minimum baseline of annual reports
        actually this method is just a smart wrapper for access spider.search() more easily
        """
        phrase = spider.phrase(f'after:{after} before:{before}')
        corps = self.search(phrase, output='cik').unique()
        print(corps)
        # 조건에 맞는 기업이 없는 경우 에러
        assert corps is not None, "There's no company matching in query"
        # min_freq 이상의 기업만 추출
        if min_freq > 0:
            corps = [corp for corp in corps if len(self.index_dict['cik'][corp]) >= min_freq]
        # get period
        if get_period:
            return [spider.phrase(f'cik:{cik} after:{after} before:{before}') for cik in corps]
        # not get period
        return [spider.phrase(f'cik:{cik}') for cik in corps]

    def get(self, key: phrase):
        """get spider.syntax object, return pd.Series of (acc, text)"""
        assert isinstance(key, spider.phrase), "keyword must be spider.syntax object"
        path = self.search(key, output='path')
        text = path.apply(lambda x: open(x, 'r').read())
        return text

    def _query_as_mod(self, mod, value):
        """ 재작성 필요 """
        date = self.index["date"]
        value = str(value)
        after = "10000000"
        before = "99999999"
        if mod == 'after':
            after = value
        elif mod == 'before':
            before = value
        elif mod == 'year':
            after = value[:4] + '0101'
            before = value[:4] + '1231'
        elif mod == 'qtr':
            after = value
            before = value[:4] + str(int(value[4:6])+3) + value[6:]
        idx = evaluate(f'({int(after)} < date) & (date < {int(before)})')
        # True to index
        return np.where(idx)[0]

    def _build_tree(self):
        if self.tree: return self.tree
        # result = ''
        # for row in self.tree.itertuples():
        #     list = ['\t'*row.depth, row.name, f"\tfiles: {row.count}\n"]
        #     result.append(''.join(list))
        return


In [40]:
files = spider(PATH)

In [41]:
a = files.company(after=20190101, min_freq=1)
b = a[0]
print(b.phrase)
c = files.get(b)

[1563536  940944 1173281 ... 1853314 1867956 1879373]
({'cik': 1563536}, {'after': '20190101', 'before': '99999999'})


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\wonhyeong\\workings\\data\\10X\\cleaned\\2020\\QTR3\\20200917_10-Q_edgar_data_1563536_0001493152-20-017944.txt'

In [35]:
index_dict = files.index_dict
index_dict['cik']['1563536']

KeyError: '1563536'

In [ ]:
class init_setting:
    def __init__(self, path):
        return
    def _date_to_path(self, date):
        date = str(date)
        year = date[:4]
        # month to quarter
        month = int(date[4:6])
        quarter = (month-1) // 3 + 1
        quarter = ''.join(['QTR', str(quarter)])
        path = self.dir / year / quarter
        return path

    def _get_tindex(self):
        """ 전부 수정 필요 """
        f = open(self.dir / 'log.txt', 'r')
        log = [l.strip() for l in f.readlines()]
        log = (l for l in log if l)

        df = pd.DataFrame(log, columns=['path'])
        df['path'] = df['path'].apply(lambda x: x.replace(str(self.dir), ''))
        df['path'] = df['path'].apply(lambda x: x[1:])

        df = df[df['path'].contains(os.sep)]
        df.reset_index(drop=True, inplace=True)

        df['acc'] = df['path'].apply(lambda x: x.split('_')[5].split('.')[0])
        df.set_index('acc', inplace=True)

        self.index = df
        return

    def _get_dir_index_faster(self):
        """
        lot more faster than os.walk
        """
        # check path is pathlib.Path
        index_path = Path(os.getcwd()) / 'index.txt'
        options = f'/MIR /FP /NC /NS /NDL /NJH /NJS /LOG:index.txt /L'
        cmd = f'robocopy {str(self.path)} NULL {options}'
        # why this is too slow????
        run(['powershell', '-c', cmd])

        f = open(index_path, 'r')
        index = [line.strip() for line in f.readlines()]

        return index

    def _get_dir_tree(self):
        tree = pd.DataFrame(columns=["name", "path", "depth", "files"])
        for root, dirs, files in os.walk(self.dir):
            name = os.path.basename(root)
            this = root.replace(self.dir, "")
            depth = int(this.count(os.sep))
            count = len(files)
            # 숨김 폴더거나, 그 자식 노드일 경우 스킵
            if any(f.startswith(".") for f in this.split(os.sep)):
                print("skip:", this)
                continue
            tree.loc[len(tree)] = [name, this, depth, count]
        return tree
    def get_index(dir):
        f = open(dir / 'log.txt', 'r')
        log = [l.strip() for l in f.readlines()]
        log = (l for l in log if l)

        df = pd.DataFrame(log, columns=['path'])
        df['path'] = df['path'].apply(lambda x: x.replace(str(dir), ''))
        df['path'] = df['path'].apply(lambda x: x[1:])

        df = df[df['path'].str.contains(os.sep, regex=False)]
        df.reset_index(drop=True, inplace=True)

        df['acc'] = df['path'].apply(lambda x: x.split('_')[5].split('.')[0])
        df.set_index('acc', inplace=True)
        return df

def get_summary(dir):
    return pd.read_csv(dir / 'summaries.csv')[['CIK', 'FILING_DATE', 'ACC_NUM', 'FORM_TYPE', 'CoName']]

def get_ticker(dir):
    ticker = pd.read_csv(dir / 'cik.csv')
    ticker.drop(ticker.columns[0], axis=1, inplace=True)
    # limit exchange to NYSE or NASDAQ
    # due to duplicated cik are common in OTC, we need to limit exchange first
    ticker.drop(ticker.index[(ticker['exchange'] != 'NYSE') & (
        ticker['exchange'] != 'Nasdaq')], inplace=True)
    # drop all duplicates
    ticker.drop_duplicates(subset='cik', keep='first', inplace=True)
    ticker.drop_duplicates(subset='name', keep='first', inplace=True)
    ticker.drop_duplicates(subset='ticker', keep='first', inplace=True)
    # column cik to index
    ticker.set_index('cik', inplace=True)
    return ticker

def findticker(cik):
    try:
        result = ticker.at[cik, 'ticker']
    except:
        return
    return result

def findexchange(cik):
    try:
        result = ticker.at[cik, 'exchange']
    except:
        return
    return result

# dir = Path(PATH)
# index = get_index(dir)
# summary = get_summary(dir)
# ticker = get_ticker(dir)

# summary['TICKER'] = summary['CIK'].map(findticker)
# summary['EXCHANGE'] = summary['CIK'].map(findexchange)


# summary['path'] = summary['FILING_DATE'].apply(str) + '_' + summary['FORM_TYPE'].apply(str) + '_' + 'edgar_data' + '_' + summary['CIK'].apply(str) + '_' + summary['ACC_NUM'].apply(str)+'.txt'
# # change column name

# summary.rename(columns={'FILING_DATE': 'date', 'FORM_TYPE': 'type', 'CoName': 'name', 'CIK': 'cik', 'TICKER': 'ticker', 'EXCHANGE':'exchange'}, inplace=True)
# summary.rename(columns={'ACC_NUM': 'acc'}, inplace=True)

# summary.set_index('acc', inplace=True)
# summary.to_pickle(dir / 'summary.pkl')
# a = pd.read_pickle(dir / 'summary.pkl')
# a